In [9]:
# Imports
import pandas as pd
import os
import matplotlib.pyplot as plt
import altair as alt
import numpy as np
import re
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output

## Read from jmeter txt File ##

In [10]:
fname = "C:/Jmeter_Dev/Results/Trust_testResults.txt"
f = open(fname)
lines = f.readlines()
rng = len(lines)
my_lt = []
for i in range(2, rng, 4):
    if i < rng:
        x = lines[i]
        my_lt.append(re.split(r'\t+', x))
    
f.close()

## Store DataFrames in List ##

In [12]:
def hms_to_sec(val):
    l = val.split(':')
    sec = int(l[1])*60 + int(l[2])
    return sec

result = []

for i in my_lt:
    trust_with_task_list = i[0].split(',')
    task_trust_dict = {val.strip().split('-')[1].strip():val.strip().split('-')[0].strip()  for val in trust_with_task_list}

    task_with_timestamp = i[1].split(',')
    task_timestamp_dict = {val.strip().split('-')[1].strip():val.strip().split('-')[0].strip()  for val in task_with_timestamp}

    df1 = pd.DataFrame(task_trust_dict.values(), index=task_trust_dict.keys())
    df1.rename(columns={0:"TrustID"}, inplace=True)

    df2 = pd.DataFrame(task_timestamp_dict.values(), index=task_timestamp_dict.keys())
    df2.rename(columns={0:"Time Taken (h:m:s)"}, inplace=True)

    final_df = df1.join(df2)

    final_df["Time(in sec)"] = final_df["Time Taken (h:m:s)"].apply(hms_to_sec)
    
    result.append(final_df)


## Helper Functions ##

In [13]:
def make_chart(df):
    selection = alt.selection_interval(encodings=['x'])

    colorCondition = alt.condition(selection, alt.value('lightgreen'), alt.value('gray'))

    x = alt.Chart(df).mark_bar().encode(
        x=alt.X('TrustID', sort='-y'),
        y='Time(in sec)',
        tooltip = ['TrustID', 'Time Taken (h:m:s)'],
        color=colorCondition
    ).interactive().add_selection(selection).properties(
        width=500,
        height=400
    )
    
    return x

def trust_frame(result_frames, trust_id):
    #trust_id = "ODART20181"
    trust_dframe = None
    for dframe in result_frames:
        trust_row = dframe[dframe['TrustID'] == trust_id]
        if trust_dframe is None:
            trust_dframe = trust_row
        else:
            trust_dframe = pd.concat([trust_dframe, trust_row], axis=0)
        
    return trust_dframe

def trust_chart(trust_frame, trust_id):
    x = alt.Chart(trust_frame.reset_index()).mark_bar().encode(
        x='index',
        y='Time(in sec)',
        color=alt.value('lightgreen'),
    ).properties(title=trust_id, width=300)
    
    return x

## Widget for Iterations ##

In [14]:
iteration_list = ['Iteration-' + str(i) for i in range(1, len(result)+1)]

w = widgets.Dropdown(
    options=iteration_list,
    value=iteration_list[0],
    description='Iteration:',
)

display(w)

def on_change(change):
    if change['name'] == 'value' and (change['new'] != change['old']):
        get_index = str(change['new']).split('-')[1]
        clear_output()
        display(w)
        make_chart(result[int(get_index)-1]).display()
        display(result[int(get_index)-1].sort_values('Time(in sec)', ascending=False))

w.observe(on_change)

Dropdown(description='Iteration:', index=3, options=('Iteration-1', 'Iteration-2', 'Iteration-3', 'Iteration-4…

alt.Chart(...)

,TrustID,Time Taken (h:m:s),Time(in sec)
130383,OMFIT20163,0:3:55,235
130387,OMFIT2019A,0:3:45,225
130391,SFT2015B,0:3:45,225
130392,SFT2017A,0:3:45,225
130379,OMFIT20153,0:2:42,162
130380,ODART20181,0:2:42,162
130381,OMFIT20181,0:2:42,162
130382,ODART20191,0:2:42,162
130388,OMFIT20201,0:2:36,156
130384,OMFIT20182,0:2:36,156


## Widget for Individual Trusts ##

In [15]:
trust_iteration_list = result[0].TrustID.tolist()

z = widgets.Dropdown(
    options=trust_iteration_list,
    value=trust_iteration_list[0],
    description='Trust:',
)

display(z)

def on_change(change):
    if change['name'] == 'value' and (change['new'] != change['old']):
        get_trust = change['new']
        clear_output()
        display(z)
        trust_df = trust_frame(result, get_trust)
        display(trust_chart(trust_df, get_trust))
        display(trust_df)

z.observe(on_change)

Dropdown(description='Trust:', options=('ODART20181', 'ODART20191', 'OMFIT20182', 'OMFIT20192', 'OMFIT20202', …

alt.Chart(...)

,TrustID,Time Taken (h:m:s),Time(in sec)
130203,ODART20181,0:3:46,226
130259,ODART20181,0:2:50,170
130324,ODART20181,0:2:45,165
130380,ODART20181,0:2:42,162
